In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/train.csv.zip')

#allow all columns to be displayed
pd.set_option('display.max_columns', None)

Now that we have the data, lets take a closer look at the data. We will look at the distribution of the data as well as the number of missing values and the type of the data.

# Evaluating Data distibution and any bad values

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
toxic_types = ['toxic',	'severe_toxic'	,'obscene',	'threat',	'insult', 	'identity_hate']

#get the value counts for each toxic type
for toxic_type in toxic_types:
    print(df[toxic_type].value_counts())
    print(df[toxic_type].value_counts(normalize=True))

0    144277
1     15294
Name: toxic, dtype: int64
0    0.904156
1    0.095844
Name: toxic, dtype: float64
0    157976
1      1595
Name: severe_toxic, dtype: int64
0    0.990004
1    0.009996
Name: severe_toxic, dtype: float64
0    151122
1      8449
Name: obscene, dtype: int64
0    0.947052
1    0.052948
Name: obscene, dtype: float64
0    159093
1       478
Name: threat, dtype: int64
0    0.997004
1    0.002996
Name: threat, dtype: float64
0    151694
1      7877
Name: insult, dtype: int64
0    0.950636
1    0.049364
Name: insult, dtype: float64
0    158166
1      1405
Name: identity_hate, dtype: int64
0    0.991195
1    0.008805
Name: identity_hate, dtype: float64


In [6]:
#find out how many of the rows have no toxic comments
print(df[toxic_types].sum(axis=1).value_counts())
df[toxic_types].sum(axis=1).value_counts(normalize=True)

0    143346
1      6360
3      4209
2      3480
4      1760
5       385
6        31
dtype: int64


0    0.898321
1    0.039857
3    0.026377
2    0.021808
4    0.011030
5    0.002413
6    0.000194
dtype: float64

As we can see above, the majority of the data has been labeled as non-toxic. We may need to be careful about the model overfitting to just label non-toxic. We will need to look at the other labels to see if they are also skewed.

In [7]:
double_toxic=df[(df.toxic==1) & (df.severe_toxic==1)]
double_toxic.shape

(1595, 8)

In [8]:
severely_toxic = df[df.severe_toxic==1]
severely_toxic.shape

(1595, 8)

In [9]:
double_toxic.equals(severely_toxic)

True

In [10]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [11]:
#get all data with obscene and insult comments
obscene_insult = df[(df.obscene==1) & (df.insult==1)]
print(obscene_insult.shape)

insult = df[df.insult==1]
insult.shape
print(insult.shape)


#get all data with obscene  comments
obscene = df[df.obscene==1]
obscene.shape

(6155, 8)
(7877, 8)


(8449, 8)

Interestingly all the ones tagged as severe_toxic are also labelled as toxic. Also a lot of the obscene and insult comments are also frequently labelled together.  Perhaps it will be useful to look at some of the examples that are labelled as only one or the other To get a better idea for what defines that category of data.

# Looking at samples within the categories to get a better idea of the data

In [12]:
#using df, get all data with obscene  comments and insult comments not equal to 1
obscene_not_insult = df[(df.obscene==1) & (df.insult!=1)]
obscene_not_insult.shape

#using df, get all data with insult  comments not equal to 1
insult_not_obscene = df[(df.obscene!=1) & (df.insult==1)]
insult_not_obscene.shape

#using df, get all data with obscene  comments and all other toxic types not equal to 1


(1722, 8)

In [13]:
#look at a few of the comments in obscene_not_insult
for item in obscene_not_insult.comment_text.head(10):
    print(item)
    print('------------------')

GET FUCKED UP. GET FUCKEEED UP.  GOT A DRINK THAT YOU CANT PUT DOWN???/ GET FUCK UP GET FUCKED UP.  I'M FUCKED UP RIGHT NOW!
------------------
My Band Page's deletion. You thought I was gone. 

Deleting the comment I posted on your 'talk page' does not delete my feelings for your hasty decision to delete my page. You, sir, are still a cock-sucking douche fuck. Sit back for a second and think about what your life has become. ............ Done? That didn't take long did it? Nope. Because, as I am most certainly aware, your life is a shitstorm of sitting in front of the computer masturbating to fictional creatures. Your attempts to get rid of me are mediocre at best. You are constantly sucking the dick of failure. You don't want a personal attack, huh? Well, too bad, Fuckcock McDickerson. You've got one. From me. You can go ahead and delete my profile from wikipedia. I'll just make another one and come right back to 'Syrthiss's talk page and insult the dick off of you. How could you shat

In [14]:
toxic_types = ['toxic',	'severe_toxic'	,'obscene',	'threat',	'insult', 	'identity_hate']
#using df, get all data with obscene  comments and all other toxic types not equal to 1

toxic_only = df[(df.toxic==1) & (df[toxic_types].sum(axis=1)==1)]
print('toxic_only shape: ', toxic_only.shape)
severe_toxic_only = df[(df.severe_toxic==1) & (df[toxic_types].sum(axis=1)==1)]
print('severe_toxic_only shape: ', severe_toxic_only.shape)

obscene_only = df[(df.obscene==1) & (df[toxic_types].sum(axis=1)==1)]
print('obscene_only shape: ', obscene_only.shape)

threat_only = df[(df.threat==1) & (df[toxic_types].sum(axis=1)==1)]
print('threat_only shape: ', threat_only.shape)

insult_only = df[(df.insult==1) & (df[toxic_types].sum(axis=1)==1)]
print('insult_only shape: ', insult_only.shape)

identity_hate_only = df[(df.identity_hate==1) & (df[toxic_types].sum(axis=1)==1)]
print('identity_hate_only shape: ', identity_hate_only.shape)

okay_only = df[(df[toxic_types].sum(axis=1)==0)]
print('okay_only shape: ', okay_only.shape)

toxic_only shape:  (5666, 8)
severe_toxic_only shape:  (0, 8)
obscene_only shape:  (317, 8)
threat_only shape:  (22, 8)
insult_only shape:  (301, 8)
identity_hate_only shape:  (54, 8)
okay_only shape:  (143346, 8)


In [15]:
for text in toxic_only.comment_text.head(10):
    print(text)
    print('------------------')

Hey... what is it..
@ | talk .
What is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?

Ask Sityush to clean up his behavior than issue me nonsensical warnings...
------------------
Bye! 

Don't look, come or think of comming back! Tosser.
------------------
I'm Sorry 

I'm sorry I screwed around with someones talk page.  It was very bad to do.  I know how having the templates on their talk page helps you assert your dominance over them.  I know I should bow down to the almighty administrators.  But then again, I'm going to go play outside....with your mom.   76.122.79.82
------------------
Why can't you believe how fat Artie is? Did you see him on his recent appearence on the Tonight Show with Jay Leno? He looks absolutely AWFUL! If I had to put money on it, I'd say that Artie Lange is a can't miss candidate for the 2007 Dead pool

nothing too clear to distinguish about this data

In [16]:
for text in obscene_only.comment_text.head(10):
    print(text)
    print('------------------')

How do you know he is dead.  Its just his plane that crashed.  Jeezz, quit busting his nuts, folks.
------------------
Bleh. I'm all out of ideas. Is there really something wrong with the server or what? The damn thing won't work for me, either Helvetica_font_new.svg (my upload of Segoe UI was actually a PNG to begin with...)
------------------
{{unblock|i said im sorry......how do u expect me to stop making accounts if u dont unblock me.....if u unblock me,ill stop making acounts...simple...........ur the ones who are making something bad to worse.....stop making a big deal...im not doing anything bad or violating anything now,so i dont get it,why am i still blocked....just get over it already.......i mean well.....im not here do violate anything.......im just trying to help.......and can someone tell alexandria to unblockme already......this block is so fucking overrated.....cant u just forgive and forget.....move on people....there are other things to worry about than a stupid block

Obscene comments seem to include heavily gaslighting, complaining, and blaming others for something they are probably responsible for.  Peoeple that exhibit these behaviors tend to narcissitic

More insights may be made but the time it takes to more closer evaluate the data before geting some sort of a bench mark doesnt not seem helpful

# Moving onto model training  

In [17]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


## a lesson learned - multilabel data split

A classic stratified split does not work with multilablled data when there are rare multilabeleed instances that only occur once in the dataset.  As a result, I found that rare data and removed it before doing the test/train split


In [18]:
"""#first we need to split the data into train and validation sets
from sklearn.model_selection import train_test_split

#split the data into train and validation sets
train, valid = train_test_split(df, test_size=0.2, random_state=42, stratify=df[toxic_types], )

print(train.shape, valid.shape)"""

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [19]:
#make a column called multi_label that is the list of all the valid labels column names
df['multi_label'] = df[toxic_types].apply(lambda x: x.index[x==1].tolist(), axis=1)


In [20]:
#if its an empty list, make it a list with one item, 'non-toxic'
df['multi_label'] = df['multi_label'].apply(lambda x: x if len(x)>0 else ['non-toxic'])

In [21]:
df.multi_label.value_counts()

[non-toxic]                                                      143346
[toxic]                                                            5666
[toxic, obscene, insult]                                           3800
[toxic, obscene]                                                   1758
[toxic, insult]                                                    1215
[toxic, severe_toxic, obscene, insult]                              989
[toxic, obscene, insult, identity_hate]                             618
[obscene]                                                           317
[insult]                                                            301
[toxic, severe_toxic, obscene, insult, identity_hate]               265
[obscene, insult]                                                   181
[toxic, severe_toxic, obscene]                                      158
[toxic, identity_hate]                                              136
[toxic, insult, identity_hate]                                  

In [22]:
condition1 = (df['toxic'] == 1) & (df['severe_toxic'] == 1) & (df['threat'] == 1) & (df['insult'] == 1)
condition2 = (df['toxic'] == 1) & (df['severe_toxic'] == 1) & (df['threat'] == 1) & (df['identity_hate'] == 1)

#find all the rows that have include conditions 1 or 2 and do not include any other toxic types
df[(condition1 | condition2) & (df[toxic_types].sum(axis=1)==4)]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,multi_label
107881,40a358e41dcc21f4,IM GOING TO KILL YOU ALL!!!!!!!!!!!!!!!!!\nI'm...,1,1,0,1,1,0,"[toxic, severe_toxic, threat, insult]"
134459,cf012f8122791d7e,DEATH TO ARABS! FREE THE JEWISH HOMELAND OF JU...,1,1,0,1,0,1,"[toxic, severe_toxic, threat, identity_hate]"


In [23]:
rare_data = df[(condition1 | condition2) & (df[toxic_types].sum(axis=1)==4)]
#drop the rare data from the df
df = df.drop(rare_data.index)

In [24]:
train, valid = train_test_split(df, test_size=0.2, random_state=42, stratify=df['multi_label'], )
#add the rare data df to the train set df
train = pd.concat([train, rare_data])


# Training a model

For base evaluation, I will start off using a simpler model

Afterwards, I will move onto training a large model, and maybe even a transformer model

NOTE:  since the competition is over I could use the labelled test set but I will continue using the validation set to simulate a realistic competition

## Base Model: Logistic Regression with multiclass using OneVsRestClassifier

The classifier appears to be doing well (.979 roc_auc), however when trying to do baseline comparisions, I found that its returning 0.5 no matter which column I want to directly compare against.  This may be indicating some lack of properly formatted data, however I am also not too comfortable with roc_auc.  As a result, I will look into it and try to evaluate a baseling again afterwards

Update: a performance of 0.5 in auc_roc means that it is no better than random guessing, which is what I was doing when testing out baseline performance.

In [36]:
train.drop('multi_label', axis=1, inplace=True)
valid.drop('multi_label', axis=1, inplace=True)

In [38]:
label_cols = ['toxic',	'severe_toxic'	,'obscene',	'threat',	'insult', 	'identity_hate']

In [41]:
#train a logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score

#instantiate the vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)

#instantiate the classifier
lr = LogisticRegression(random_state=1,  max_iter=1000,)

#instantiate the pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', OneVsRestClassifier(lr))])

#fit the model
clf.fit(train['comment_text'], train[label_cols])


Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.7, stop_words='english')),
                ('clf', OneVsRestClassifier(estimator=LogisticRegression()))])

In [72]:
#make predictions on the validation set
preds = clf.predict_proba(valid['comment_text'])

roc_auc_score(valid[label_cols], preds)

0.9794619893684119

### creating random-guess baselines

In [87]:
#make a copy of preds, 
random_guess = preds.copy()
#set all values to 0
random_guess[:]=0
#set the first item in the array to 1 for each row
random_guess[:,5]=1
roc_auc_score(valid[label_cols], random_guess)

0.5

In [99]:
import numpy as np
#make a list of arrays that is the same length as the validation set
random_guesses = [np.array([0.6]) for i in range(len(valid))]

In [100]:
valid[label_cols[1]].value_counts()

0    31595
1      319
Name: severe_toxic, dtype: int64

In [102]:
roc_auc_score(valid[label_cols[0]],random_guesses)

0.5

### getting test set to evaluate baseline model

In [103]:
test = pd.read_csv('../data/test.csv.zip')
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [104]:
test_preds = clf.predict_proba(test['comment_text'])
#put the predictions into test dataframe
test[label_cols] = test_preds
#drop the comment_text column
test.drop('comment_text', axis=1, inplace=True)
#save the test dataframe to a csv file
test.to_csv('../data/submission1_LR.csv', index=False)

In [105]:
test.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.992855,0.124077,0.990072,0.021650,0.894809,0.158616
1,0000247867823ef7,0.013340,0.003690,0.008333,0.001904,0.012149,0.004024
2,00013b17ad220c46,0.046065,0.004860,0.018429,0.001936,0.021711,0.005639
3,00017563c3f7919a,0.003922,0.002305,0.003563,0.001111,0.003901,0.001111
4,00017695ad8997eb,0.040152,0.002774,0.010028,0.002090,0.013592,0.003268


### evaluating on the test set

In [ ]:
preds = clf.predict_proba(valid['comment_text'])

## Base Model runs 2 and 3: Random Forest with multiclass using OneVsRestClassifier, with max_df=1 and max_df=0.5

Note: max_df=1 led to a steep decline in results whereas 0.5 got the same results as 0.7 in run 1

### Run 2

In [109]:
#instantiate the vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_df=1)

#instantiate the classifier
lr = LogisticRegression(random_state=1,  max_iter=1000,)

#instantiate the pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', OneVsRestClassifier(lr))])

#fit the model
clf.fit(train['comment_text'], train[label_cols])

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=1, stop_words='english')),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000,
                                                                  random_state=1)))])

In [110]:
#make predictions on the validation set
preds = clf.predict_proba(valid['comment_text'])

roc_auc_score(valid[label_cols], preds)

0.5492175306147505

### Run 3

In [111]:
#instantiate the vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_df=0.5)

#instantiate the classifier
lr = LogisticRegression(random_state=1,  max_iter=1000,)

#instantiate the pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', OneVsRestClassifier(lr))])

#fit the model
clf.fit(train['comment_text'], train[label_cols])

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.5, stop_words='english')),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000,
                                                                  random_state=1)))])

In [112]:
#make predictions on the validation set
preds = clf.predict_proba(valid['comment_text'])

roc_auc_score(valid[label_cols], preds)

0.9794619893684119

In [ ]:
test = pd.read_csv('../data/test.csv.zip')

test_preds = clf.predict_proba(test['comment_text'])
#put the predictions into test dataframe
test[label_cols] = test_preds
#drop the comment_text column
test.drop('comment_text', axis=1, inplace=True)
#save the test dataframe to a csv file
test.to_csv('../data/submission3_LR.csv', index=False)

## Baseline 3: OVR with RF

With a score of .933, It doesnt seem to do as well as LR-OVR

In [117]:
#instantiate the vectorizer
tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)

#instantiate the random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1, n_estimators=100, max_depth=10)

#instantiate the pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', OneVsRestClassifier(rf))])

#fit the model
train_copy = train.copy() # create a copy of the train dataframe
clf.fit(train_copy['comment_text'], train_copy[label_cols])


Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.7, stop_words='english')),
                ('clf',
                 OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                                      random_state=1)))])

In [118]:
#make predictions on the validation set
preds = clf.predict_proba(valid['comment_text'])

roc_auc_score(valid[label_cols], preds)

0.9330115931912687

## attempt 2: Spacy classifier

In [119]:
#train a one vs the rest spacy model
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
from spacy.lang.en import English

#create a tokenizer function
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

def spacy_tokenizer(sentence):
    #create token object, which is used to create documents with linguistic annotations
    mytokens = parser(sentence)

    #lemmatize each token and convert each token into lowercase
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]

    #remove stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    #return preprocessed list of tokens
    return mytokens

#instantiate the vectorizer
tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer, max_df=0.7)

#instantiate the random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1, n_estimators=100, max_depth=10)

#instantiate the pipeline
clf = Pipeline([('tfidf', tfidf), ('clf', OneVsRestClassifier(rf))])

#fit the model
train_copy = train.copy() # create a copy of the train dataframe

clf.fit(train_copy['comment_text'], train_copy[label_cols])

#make predictions on the validation set
preds = clf.predict_proba(valid['comment_text'])

roc_auc_score(valid[label_cols], preds)

In [27]:
# train using a spacy model for multilabel text classification
import spacy
import random
from spacy.util import minibatch, compounding
from spacy import displacy
from spacy.lang.en import English
from spacy.pipeline import TextCategorizer

In [34]:
import spacy
from spacy.pipeline.textcat import Config, single_label_cnn_config
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import random
from spacy.training.example import Example

# Load a pre-trained SpaCy model
nlp = spacy.load("en_core_web_sm")

# Define your text data and associated labels
data = [
    ("This is a positive sentence.", ["positive"]),
    ("This is another positive sentence.", ["positive"]),
    ("This is a negative sentence.", ["negative"]),
    ("This is a mixed sentence.", ["positive", "negative"]),
]

# Split data into text and labels
texts, labels = zip(*data)

# Initialize a SpaCy text classifier with multi-label support
config = Config().from_str(single_label_cnn_config)
config['threshold'] = 0.5  # Adjust this threshold for label assignment

text_classifier = nlp.add_pipe("textcat", config=config, last=True)
text_classifier.add_label("positive")
text_classifier.add_label("negative")

# Convert labels into a multi-label format
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(labels)

# Convert the entire training data into Example objects
train_examples = []

for text, annotations in zip(texts, binary_labels):
    example = Example.from_dict(nlp.make_doc(text), {"cats": annotations})
    train_examples.append(example)

# Train the text classifier with the entire dataset
for epoch in range(10):  # You can adjust the number of training epochs
    losses = {}
    random.shuffle(train_examples)
    nlp.update(train_examples, drop=0.5, losses=losses)
    print(losses)

# Test the model on new text data
test_data = ["This is a positive sentence.", "This is a negative sentence.", "This is a mixed sentence."]
for text in test_data:
    doc = nlp(text)
    predicted_labels = [label for label, score in doc.cats.items() if score >= 0.5]
    print(f"Text: {text}\nPredicted Labels: {predicted_labels}\n")

# Evaluate the model (optional)
# You can use metrics like accuracy, precision, recall, and F1-score for evaluation
# Note that in multi-label classification, traditional accuracy may not be the best metric
# You may want to look at precision, recall, and F1-score for each label separately


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [29]:
#load the spacy model
nlp = spacy.load('en_core_web_sm')

#add the textcat component to the pipeline
textcat = nlp.create_pipe("textcat", config={"exclusive_classes": False, "architecture": "simple_cnn"})
nlp.add_pipe(textcat, last=True)

#add the labels to the textcat component
textcat.add_label('toxic')
textcat.add_label('severe_toxic')
textcat.add_label('obscene')
textcat.add_label('threat')
textcat.add_label('insult')
textcat.add_label('identity_hate')
textcat.add_label('non-toxic')

#convert the train data into a list of tuples
train_data = list(train.apply(lambda x: (x['comment_text'], {'cats': {'toxic': x['toxic'], 'severe_toxic': x['severe_toxic'], 'obscene': x['obscene'], 'threat': x['threat'], 'insult': x['insult'], 'identity_hate': x['identity_hate'], 'non-toxic': x['non-toxic']}}), axis=1))

#convert the validation data into a list of tuples
valid_data = list(valid.apply(lambda x: (x['comment_text'], {'cats': {'toxic': x['toxic'], 'severe_toxic': x['severe_toxic'], 'obscene': x['obscene'], 'threat': x['threat'], 'insult': x['insult'], 'identity_hate': x['identity_hate'], 'non-toxic': x['non-toxic']}}), axis=1))


ConfigValidationError: 

Config validation error
textcat -> architecture	extra fields not permitted
textcat -> exclusive_classes	extra fields not permitted
{'nlp': <spacy.lang.en.English object at 0x0000024E65560FA0>, 'name': 'textcat', 'architecture': 'simple_cnn', 'exclusive_classes': False, 'model': {'@architectures': 'spacy.TextCatEnsemble.v2', 'linear_model': {'@architectures': 'spacy.TextCatBOW.v2', 'exclusive_classes': True, 'ngram_size': 1, 'no_output_layer': False}, 'tok2vec': {'@architectures': 'spacy.Tok2Vec.v2', 'embed': {'@architectures': 'spacy.MultiHashEmbed.v2', 'width': 64, 'rows': [2000, 2000, 1000, 1000, 1000, 1000], 'attrs': ['ORTH', 'LOWER', 'PREFIX', 'SUFFIX', 'SHAPE', 'ID'], 'include_static_vectors': False}, 'encode': {'@architectures': 'spacy.MaxoutWindowEncoder.v2', 'width': 64, 'window_size': 1, 'maxout_pieces': 3, 'depth': 2}}}, 'scorer': {'@scorers': 'spacy.textcat_scorer.v1'}, 'threshold': 0.5, '@factories': 'textcat'}

In [ ]:

#convert the test data into a list of tuples
#test_data = list(test.apply(lambda x: (x['comment_text'], {'cats': {'toxic': x['toxic'], 'severe_toxic': x['severe_toxic'], 'obscene': x['obscene'], 'threat': x['threat'], 'insult': x['insult'], 'identity_hate': x['identity_hate'], 'non-toxic': x['non-toxic']}}), axis=1))

#train the model
n_iter=10
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']

with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    print("Training model...")
    for i in range(n_iter):
        losses = {}
        random.shuffle(train_data)
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
        print("Losses", losses)

: 

In [ ]:
#evaluate the model
def evaluate(model, data):
    scorer = nlp.evaluate(data)
    print('Losses', scorer['textcat_loss'])
    print('Accuracy', scorer['textcat_acc'])
    return scorer

evaluate(nlp, valid_data)

: 

# Attempt 3: huggingface

After doing some research, it is clear that we can also use huggingface to train a model, outputting the probabilities and then performing auc-roc